In [ ]:
!git clone https://github.com/kingfengji/mGBDT.git

Cloning into 'mGBDT'...
remote: Enumerating objects: 91, done.
remote: Total 91 (delta 0), reused 0 (delta 0), pack-reused 91
Unpacking objects: 100% (91/91), done.


In [ ]:
import sys
sys.path.insert(0, "/content/mGBDT/lib")

In [ ]:
!pip install scikit-optimize

     |████████████████████████████████| 81kB 1.9MB/s 


In [ ]:
import pandas as pd
import numpy as np
from numpy import mean
import os
import time

from mgbdt import MGBDT, MultiXGBModel

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve
from sklearn.preprocessing import LabelBinarizer

from scipy import interp

from skopt.space import Integer
from skopt.space import Real
from skopt.utils import use_named_args
from skopt import gp_minimize


In [ ]:
files_path = '/path/to/datasets'

In [ ]:
files_list = os.listdir(files_path)

# Data preparation

In [ ]:
def read_dataset(folder_path, filename):
    df = pd.read_csv(os.path.join(folder_path, filename))
    return df

In [ ]:
def dummy_encode(df):
    cols_to_encode = list(df.select_dtypes(include=['category','object']))
    if len(cols_to_encode): 
        df = pd.get_dummies(df, columns = cols_to_encode, prefix=cols_to_encode)

    return df.values #### df

In [ ]:
def prepare_data(df):
    df = df.fillna(df.mean())
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    lb = LabelEncoder()
    y = lb.fit_transform(y)
    X = dummy_encode(X)
    # print('X', X)
    classes = np.unique(y)

    return X, y, classes

In [ ]:
def create_model(X_train, num_classes, max_depth, num_boost_round):

    net = MGBDT(loss="CrossEntropyLoss", target_lr=1.0, epsilon=0.1)
    net.add_layer("tp_layer",
        F=MultiXGBModel(input_size=X_train.shape[1], output_size=5, learning_rate=0.1, max_depth=5, num_boost_round=num_boost_round),
        G=None)
    net.add_layer("tp_layer",
        F=MultiXGBModel(input_size=5, output_size=num_classes, learning_rate=0.1, max_depth=max_depth, num_boost_round=num_boost_round),
        G=MultiXGBModel(input_size=num_classes, output_size=5, learning_rate=0.1, max_depth=max_depth, num_boost_round=num_boost_round))
    
    # init the forward mapping
    net.init(X_train, n_rounds=num_boost_round)

    return net

# Metrics calculations

In [ ]:
def metrics_calc(y_true, y_pred, classes):

    y_pred_arg = np.argmax(y_pred, axis=1)
    y_true_label = label_binarize(y_true, classes)
    y_pred_soft = softmax(y_pred)

    cnf_matrix = confusion_matrix(y_true, y_pred_arg)

    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)  
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)

    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)

    TPR = TP/(TP+FN)
    FPR = FP/(FP+TN)
    PPV = TP/(TP+FP)
    ACC = (TP+TN)/(TP+FP+FN+TN)

    mean_tpr = np.nanmean(TPR)
    mean_fpr = np.nanmean(FPR)
    mean_precision = np.nanmean(PPV)

    precision = dict()
    recall = dict()
    fpr = dict()
    tpr = dict()
    roc_auc_list = []
    pr_auc_list = []

    # different calculations if it is binary classification or multi-class
    if len(classes) == 2:
        fpr, tpr, threshold_roc = roc_curve(y_true_label, y_pred_soft[:, 1])
        roc_auc = auc(fpr, tpr)
        roc_auc_list.append(roc_auc)

        
        precision, recall, threshold_pr = precision_recall_curve(y_true_label,
                                                            y_pred_soft[:, 1])
        
        pr_auc = auc(recall, precision)

        pr_auc_list.append(pr_auc)

    else:
        for i in range(y_pred.shape[1]):
            fpr[i], tpr[i], threshold_roc = roc_curve(y_true_label[:, i], y_pred_soft[:, i])
            roc_auc = auc(fpr[i], tpr[i])
            roc_auc_list.append(roc_auc)

            precision[i], recall[i], threshold_pr = precision_recall_curve(y_true_label[:, i],
                                                                y_pred_soft[:, i])

            pr_auc = auc(recall[i], precision[i])
            pr_auc_list.append(pr_auc)

    mean_roc_auc = np.nanmean(roc_auc_list)
    mean_pr_auc = np.nanmean(pr_auc_list)

    return mean_tpr, mean_fpr, mean_precision, mean_roc_auc, mean_pr_auc


In [ ]:
def softmax(x):
    """Compute softmax values"""
    return np.exp(x) / np.sum(np.exp(x), axis=1, keepdims=True)

# Training

In [ ]:
for file in files_list:
    print('file', file)

    epsilon=0.1

    df = read_dataset(files_path, file)
    X, y, classes = prepare_data(df)

    search_space = [Integer(1, 8, name='num_boost_round'),
                    Integer(2, 7, name='max_depth')]

    @use_named_args(search_space)
    def evaluate_model(**params):
        results = []
        kf = KFold(n_splits=3, random_state=42, shuffle=True)
        kf.get_n_splits(X)

        for train_index, test_index in kf.split(X, y):

            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            model = create_model(X, len(classes), **params)

            model.fit(X_train, y_train, n_epochs=1, eval_metric="accuracy")
            y_proba = model.forward(X_test)

            result = accuracy_score(y_test, y_proba.argmax(axis=1))
            results.append(result)


        # calculate the mean of the scores
        estimate = mean(results)
        return 1.0 - estimate

    result = gp_minimize(evaluate_model, search_space, n_calls=50)

    max_depth, num_boost_round = result.x

    kf = KFold(n_splits=10, random_state=42, shuffle=True)
    kf.get_n_splits(X)

    acc_list = []
    tpr_list = []
    fpr_list = []
    precision_list = []
    roc_auc_list = []
    pr_auc_list = []
    training_time_list = []
    inference_time_list = []
    cross_val = []

    for fold_num, data_index in enumerate(kf.split(X, y)):
        train_index, test_index = data_index[0], data_index[1]
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        model = create_model(X_train, len(classes), max_depth, num_boost_round)
        t0 = time.time()
        model.fit(X_train, y_train, n_epochs=8,
                eval_sets=[(X_test, y_test)],
                eval_metric="accuracy")

        t1 = time.time()
        train_time = t1 - t0

        t0 = time.time()
        y_proba = model.forward(X_test)
        t1 = time.time()

        inference_time = (t1 - t0) * 1000 / y_test.shape[0]
        print('inference_time', inference_time)

        mean_tpr, mean_fpr, mean_precision, mean_roc_auc, mean_pr_auc = metrics_calc(y_test, y_proba, classes)
        acc = accuracy_score(y_test, y_proba.argmax(axis=1))

        acc_list.append("{:.3f}".format(acc))
        tpr_list.append("{:.3f}".format(mean_tpr))
        fpr_list.append("{:.3f}".format(mean_fpr))
        precision_list.append("{:.3f}".format(mean_precision))
        roc_auc_list.append("{:.3f}".format(mean_roc_auc))
        pr_auc_list.append("{:.3f}".format(mean_pr_auc))
        training_time_list.append("{:.1f}".format(train_time))
        inference_time_list.append("{:.1f}".format(inference_time))

        cross_val.append(fold_num + 1)


    results_dict = {'Dataset Name':[file.split('.')[0]] * 10,
                'Algorithm Name':['mGBDT'] * 10,
                'Cross Validation':cross_val,
                'Hyper Parameters Values': [[max_depth, num_boost_round]] * 10,
                'Accuracy':acc_list,
                'TPR':tpr_list,
                'FPR':fpr_list,
                'Precision':precision_list,
                'AUC':roc_auc_list,
                'PR-Curve':pr_auc_list,
                'Training Time':training_time_list,
                'Inference Time':inference_time_list
                }

    df = pd.DataFrame.from_dict(results_dict)

    df.to_csv('PATH/TO/FILE',
              mode='a', header=False)


file analcatdata_lawsuit.csv
file baseball.csv
file analcatdata_boxing1.csv
file analcatdata_asbestos.csv
file bodyfat.csv
file autos.csv
file analcatdata_broadwaymult.csv
file chscase_vine1.csv
file braziltourism.csv
file analcatdata_germangss.csv
file fri_c0_250_5.csv
file lowbwt.csv
file cloud.csv
file chatfield_4.csv
file diabetes.csv
file diggle_table_a2.csv
file meta.csv
file disclosure_z.csv
file lupus.csv
file kc3.csv
file no2.csv
file pm10.csv
file kidney.csv
file labor.csv
file elusage.csv
file mfeat-karhunen.csv
file newton_hema.csv
file plasma_retinol.csv
file rmftsa_sleepdata.csv
file mfeat-morphological.csv
file schizo.csv
file squash-unstored.csv
file socmob.csv
file sleuth_case2002.csv
file prnn_synth.csv
file squash-stored.csv
file solar-flare.csv
file schlvote.csv
file teachingAssistant.csv
file tae.csv
file visualizing_livestock.csv
file rabe_131.csv
file transplant.csv
file triazines.csv
file veteran.csv
file white-clover.csv
file vote.csv
file acute-inflammation.cs

/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:614: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase memory consumption")
[ 2020-08-20 08:40:40,671][mgbdt.log] [epoch=0/1][train] loss=1.924872, score=0.223881
[ 2020-08-20 08:40:41,947][mgbdt.log] [epoch=1/1][train] loss=1.667886, score=0.582090
[ 2020-08-20 08:40:42,780][mgbdt.log] [epoch=0/1][train] loss=1.979181, score=0.119403
[ 2020-08-20 08:40:44,023][mgbdt.log] [epoch=1/1][train] loss=1.713784, score=0.761194
[ 2020-08-20 08:40:44,853][mgbdt.log] [epoch=0/1][train] loss=2.003180, score=0.132353
[ 2020-08-20 08:40:46,109][mgbdt.log] [epoch=1/1][train] loss=1.671446, score=0.794118
[

hyper params time 330.0525507926941


/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:614: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase memory consumption")
[ 2020-08-20 08:46:10,566][mgbdt.log] [epoch=0/8][train] loss=1.972244, score=0.077778
[ 2020-08-20 08:46:10,783][mgbdt.log] [epoch=0/8][test] loss=1.978859, score=0.000000
[ 2020-08-20 08:46:12,023][mgbdt.log] [epoch=1/8][train] loss=1.850185, score=0.366667
[ 2020-08-20 08:46:12,245][mgbdt.log] [epoch=1/8][test] loss=1.861167, score=0.181818
[ 2020-08-20 08:46:13,489][mgbdt.log] [epoch=2/8][train] loss=1.775830, score=0.688889
[ 2020-08-20 08:46:13,731][mgbdt.log] [epoch=2/8][test] loss=1.825427, score=0.636364
[ 20

inference_time 19.272522492842242


...

inference_time 21.142864227294922


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_ranking.py:808: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_ranking.py:677: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:125: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:126: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
/usr/local/lib/python3.

inference_time 21.68116569519043


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:614: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase memory consumption")
[ 2020-08-20 09:04:27,426][mgbdt.log] [epoch=0/8][train] loss=0.687820, score=0.556701
[ 2020-08-20 09:04:27,641][mgbdt.log] [epoch=0/8][test] loss=0.717996, score=0.400000
[ 2020-08-20 09:04:28,878][mgbdt.log] [epoch=1/8][train] loss=0.624715, score=0.783505
[ 2020-08-20 09:04:29,092][mgbdt.log] [epoch=1/8][test] loss=0.678946, score=0.700000
[ 2020-08-20 09:04:30,343][mgbdt.log] [epoch=2/8][train

inference_time 21.842646598815918
